In [1]:
!pip install supabase pandas python-dotenv pydantic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.2 MB/s eta 0:00:00


In [5]:
!mv supabase.env .env


## ServQuick data

In [19]:
import os
import pandas as pd
import uuid
from supabase import create_client, Client
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables
load_dotenv(".env")

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

if not SUPABASE_URL or not SUPABASE_KEY:
    raise ValueError("Supabase credentials are missing. Check your .env file.")

# Initialize Supabase client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# Function to insert unique customers
def insert_customer(customer):
    existing_customer = supabase.table("Customers").select("*").eq("phone_number", customer["phone_number"]).execute()
    if not existing_customer.data:
        customer_id = str(uuid.uuid4())  # Generate a unique ID for the customer
        customer["customer_id"] = customer_id

        # Ensure all values in the customer dictionary are valid JSON types
        for key, value in customer.items():
            if pd.isnull(value):  # Check for NaN values
                customer[key] = None  # Replace NaN with None
            elif isinstance(value, float) and (pd.isna(value) or value == float('inf') or value == float('-inf')):
                customer[key] = None  # Replace infinite or NaN floats with None
            # Add other checks and conversions for potential invalid values

        supabase.table("Customers").insert(customer).execute()
        return customer_id
    return existing_customer.data[0]["customer_id"]

# Function to insert orders
def insert_order(order):
    supabase.table("Orders").insert(order).execute()

# Read and process the Excel file
def process_excel(file_path):
    # Read the Excel file
    data = pd.read_excel(file_path)

    column_map = {
        "Customer name": "Customer name",
        "Customer mobile": "Customer mobile",
        "Customer email": "Customer email",
        "Customer address": "Customer address",
        "Sale date": "Sale date",
        "Receipt no": "Receipt no",
        "Ordertype name": "Ordertype name",
        "Item name": "Item name",
        "Variant name": "Variant name",
        "Selling price": "Selling price",
        "Item quantity": "Item quantity",
        "Item amount": "Item amount",
    }

    # Ensure the necessary columns exist
    required_columns = column_map.keys()
    for col in required_columns:
        if col not in data.columns:
            raise ValueError(f"Missing required column: {col}")

    # Convert numeric columns to proper types
    data["Item quantity"] = pd.to_numeric(data["Item quantity"], errors="coerce")
    data["Item amount"] = pd.to_numeric(data["Item amount"], errors="coerce")

    # Group items by receipt ID
    grouped = data.groupby("Receipt no").apply(lambda group: {
        "order_items": group.apply(lambda row: {
            "item_name": row["Item name"],
            "quantity": row["Item quantity"],
            "amount": row["Item amount"],
        }, axis=1).tolist(),
        "order_items_text": "; ".join(
            f'{row["Item name"]} (x{row["Item quantity"]})' for _, row in group.iterrows()
        ),
    }).reset_index(name="grouped_data")

    # Extract grouped data into a DataFrame
    grouped_data = pd.json_normalize(grouped["grouped_data"])
    grouped = pd.concat([grouped, grouped_data], axis=1)

    # Merge grouped data back with the original dataset
    final_data = pd.merge(data.drop_duplicates("Receipt no"), grouped, on="Receipt no", how="left")

    # Process customers and orders
    for _, row in final_data.iterrows():
        # Extract customer details
        customer = {
            "name": row[column_map["Customer name"]],
            "phone_number": row[column_map["Customer mobile"]],
            "email": row[column_map["Customer email"]],
            "address": row[column_map["Customer address"]],
        }

        # Insert the customer and retrieve their ID
        customer_id = insert_customer(customer)

        # Skip if customer insertion fails
        if not customer_id:
            continue


        # Prepare order details
        order = {
            "order_id": str(uuid.uuid4()),  # Generate a unique ID for the order
            "customer_id": customer_id,
            "order_date": row[column_map["Sale date"]].isoformat()
            if isinstance(row[column_map["Sale date"]], pd.Timestamp)
            else str(row[column_map["Sale date"]]),
            "order_items": row["order_items"],
            "order_items_text": row["order_items_text"],
            "total_amount": sum(item["amount"] for item in row["order_items"]),
            "order_type": row[column_map["Ordertype name"]],
        }


        # Insert the order
        insert_order(order)

    print("Data successfully inserted into Supabase.")

# Example usage
if __name__ == "__main__":
    # Provide the path to your Excel file
    excel_file_path = "/content/Sales_deta_by_receipt_this_year.xls"  # Replace with your file path
    process_excel(excel_file_path)


<ipython-input-19-58295455a370>:74: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = data.groupby("Receipt no").apply(lambda group: {


APIError: {'code': '22P02', 'details': None, 'hint': None, 'message': 'invalid input value for enum order_type: "Eat in"'}

In [12]:
import os
import pandas as pd
import uuid
from supabase import create_client, Client
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables
load_dotenv(".env")

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

if not SUPABASE_URL or not SUPABASE_KEY:
    raise ValueError("Supabase credentials are missing. Check your .env file.")

# Initialize Supabase client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# Function to insert unique customers
def insert_customer(customer):
    existing_customer = supabase.table("Customers").select("*").eq("phone_number", customer["phone_number"]).execute()
    if not existing_customer.data:
        customer_id = str(uuid.uuid4())  # Generate a unique ID for the customer
        customer["customer_id"] = customer_id

        # Ensure all values in the customer dictionary are valid JSON types
        for key, value in customer.items():
            if pd.isnull(value):  # Check for NaN values
                customer[key] = None  # Replace NaN with None
            elif isinstance(value, float) and (pd.isna(value) or value == float('inf') or value == float('-inf')):
                customer[key] = None  # Replace infinite or NaN floats with None

        supabase.table("Customers").insert(customer).execute()
        return customer_id
    return existing_customer.data[0]["customer_id"]

# Function to insert unique orders
def insert_order(order):
    # Check for existing order using receipt_id
    supabase.table("Orders").insert(order).execute()
    # existing_order = supabase.table("Orders").select("*").eq("receipt_id", order["receipt_id"]).execute()
    # if not existing_order.data:
    #     supabase.table("Orders").insert(order).execute()

def process_excel(file_path):
    # Read the Excel file
    data = pd.read_excel(file_path)

    column_map = {
        "Customer name": "Customer name",
        "Customer mobile": "Customer mobile",
        "Customer email": "Customer email",
        "Customer address": "Customer address",
        "Sale date": "Sale date",
        "Receipt no": "Receipt no",  # Used for grouping but not stored in Supabase
        "Ordertype name": "Ordertype name",
        "Item name": "Item name",
        "Variant name": "Variant name",
        "Selling price": "Selling price",
        "Item quantity": "Item quantity",
        "Item amount": "Item amount",
    }

    # Ensure the necessary columns exist
    required_columns = column_map.keys()
    for col in required_columns:
        if col not in data.columns:
            raise ValueError(f"Missing required column: {col}")

    # Convert numeric columns to proper types
    data["Item quantity"] = pd.to_numeric(data["Item quantity"], errors="coerce")
    data["Item amount"] = pd.to_numeric(data["Item amount"], errors="coerce")

    # Group items by receipt number (used only for grouping here)
    grouped = data.groupby("Receipt no").apply(lambda group: {
        "order_items": group.apply(lambda row: {
            "item_name": row["Item name"],
            "quantity": row["Item quantity"],
            "amount": row["Item amount"],
        }, axis=1).tolist(),
        "order_items_text": "; ".join(
            f'{row["Item name"]} (x{row["Item quantity"]})' for _, row in group.iterrows()
        ),
    }).reset_index(name="grouped_data")

    # Extract grouped data into a DataFrame
    grouped_data = pd.json_normalize(grouped["grouped_data"])
    grouped = pd.concat([grouped, grouped_data], axis=1)

    # Merge grouped data back with the original dataset
    final_data = pd.merge(data.drop_duplicates("Receipt no"), grouped, on="Receipt no", how="left")

    # Define a mapping for valid order types
    order_type_map = {
        "Dine-In": "Dine-In",
        "Delivery": "Delivery",
        "Takeaway": "Take away",  # Map "Takeaway" to "Take away"
        "Take away": "Take away",  # Allow direct match
        "Eat in": "Dine-In",       # Handle alternate naming
    }

    # Process customers and orders
    for _, row in final_data.iterrows():
        # Extract customer details
        customer = {
            "name": row[column_map["Customer name"]],
            "phone_number": row[column_map["Customer mobile"]],
            "email": row[column_map["Customer email"]],
            "address": row[column_map["Customer address"]],
        }

        # Insert the customer and retrieve their ID
        customer_id = insert_customer(customer)

        # Skip if customer insertion fails
        if not customer_id:
            continue

        # Get and validate the order type
        order_type = row[column_map["Ordertype name"]]
        if order_type not in order_type_map:
            print(f"Skipping order with invalid order type: {order_type}")
            continue

        # Prepare order details
        order = {
            "order_id": str(uuid.uuid4()),  # Generate a unique ID for the order
            "customer_id": customer_id,
            "order_date": row[column_map["Sale date"]].isoformat()
            if isinstance(row[column_map["Sale date"]], pd.Timestamp)
            else str(row[column_map["Sale date"]]),
            "order_items": row["order_items"],
            "order_items_text": row["order_items_text"],
            "total_amount": sum(item["amount"] for item in row["order_items"]),
            "order_type": order_type_map[order_type],  # Map to a valid enum type
        }

        # Insert the order
        try:
            insert_order(order)
        except Exception as e:
            print(f"Failed to insert order: {e}")

    print("Data successfully inserted into Supabase.")


# Example usage
if __name__ == "__main__":
    # Provide the path to your Excel file
    excel_file_path = "/content/Sales_deta_by_receipt_last_year.xls"  # Replace with your file path
    process_excel(excel_file_path)


<ipython-input-12-4f0120778178>:76: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = data.groupby("Receipt no").apply(lambda group: {


APIError: {'code': '42501', 'details': None, 'hint': None, 'message': 'new row violates row-level security policy for table "Customers"'}

## Reservation data


In [14]:
import pandas as pd
from supabase import create_client, Client
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv(".env")

# Supabase configuration
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# Load the reservation data
# reservation_file = "Reservation_Details.xlsx"
# df = pd.read_excel(reservation_file)
df = pd.read_excel("/content/Reservation list update (1).xlsx", sheet_name="Update reservation Sheet")

# Clean up and normalize the data
df.columns = df.columns.str.strip()  # Remove extra spaces from column names
df["Total Bill"] = pd.to_numeric(df["Total Bill"].str.replace(",", "").str.replace(" BDT", ""), errors='coerce')  # Clean bill values
df["Company Name"] = df["Company Name"].fillna("")
df["Mobile Number"] = df["Mobile Number"].astype(str)

# Process each reservation row
for _, row in df.iterrows():
    phone_number = row["Mobile Number"]
    is_returning = row["Return Guest"] == "Yes"
    is_vip = row["Total Bill"] > 30000 # and ("family" not in row["Company Name"].lower() and "friends" not in row["Company Name"].lower())
    company_name = row["Company Name"] if ("family" not in row["Company Name"].lower() and "friend" not in row["Company Name"].lower()) else None

    # Lookup customer by phone number in Supabase
    response = supabase.table("Customers").select("*").eq("phone_number", phone_number).execute()

    if response.data:
        # Update existing customer
        customer_id = response.data[0]["customer_id"]
        update_data = {
            "is_returning_customer": is_returning,
            "is_VIP": is_vip
        }
        # Update company name if applicable
        if company_name:
            update_data["company_name"] = company_name

        supabase.table("Customers").update(update_data).eq("customer_id", customer_id).execute()
    else:
        # Optionally create a new customer if the phone number doesn't exist
        print(f"Customer with phone number {phone_number} not found in Supabase.")
        # Uncomment to add the customer
        supabase.table("Customers").insert({
            "phone_number": phone_number,
            "name": row["Guest Name"],
            "company_name": company_name if company_name else "",
            "is_returning_customer": is_returning,
            "is_VIP": is_vip
        }).execute()

print("Update process completed!")


Customer with phone number nan not found in Supabase.


APIError: {'code': 'PGRST102', 'details': None, 'hint': None, 'message': 'Empty or invalid json'}